<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900" style="text-align:center;">**Notebook 6:** 嵌入模型和语意推理</font>

<br>

在前面的 notebook 中，我们开始处理大型文档，学习了用相同的技术来处理多种内容！在这个过程中，我们注意到一些现有技术仍无法解决的动态解释（on-the-fly interperation）中的挑战。我们将在这个 notebook 讨论另一个方法：**嵌入模型**（embedding models）。

<br>  

### **学习目标：**

* 熟悉嵌入（单词、短语或文档的数字化表示），以及它们如何帮助深度学习模型处理语义。
* 了解如何在大规模文档处理中应用嵌入模型，增强我们现有的文档摘要和知识提取方法。

<br> 

### **思考问题：**

* 嵌入如何有助于更深入地理解文档块，尤其是在处理不一致或文本转换问题的时候？
* 怎么用嵌入模型在细节和计算效率之间取得平衡？能不能进行 LLM 重述（rephrase）或规范化您的问题？或者，我们能否针对任务微调？
* 嵌入模型如何补充我们之前开发的运行状态链和知识库？*（下一个 notebook 会讲到）*

<br>  

### **Notebook 版权声明：**

* 本 notebook 是 [**NVIDIA 深度学习培训中心**](https://www.nvidia.cn/training/)的课程[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)中的一部分，未经 NVIDIA 授权不得分发。

<br> 

### **环境设置：**


In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -qq langchain langchain-nvidia-ai-endpoints gradio
# %pip install -qq arxiv pymupdf

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

----

<br>

## **第 1 部分：** 重温嵌入模型

我们将在本节回顾用深度学习做自然语言处理的思想，并定义什么是嵌入模型，以及它们是怎么跟我们理所当然使用的工具相关联。

<br>

### **了解隐空间嵌入（Latent Embeddings）**

隐空间嵌入是深度学习网络中的桥梁，跨越了输入和输出之间的鸿沟。我们考虑一个用来分类 [MNIST 数字](https://en.wikipedia.org/wiki/MNIST_database)的，只有两层的轻量网络。为此，输入和输出分别是展开的（flattened）图像和独热（one-hot）概率向量。其第一层生成的值是图像的隐嵌入，优化后会收敛为对最后一层有用的表示。这使它们成为**有丰富语义的嵌入**，虽然不是人类可读的，但仍然可以利用其原始的向量性质。

<br>

### **词嵌入：语言模型的基础**

词嵌入是单个词的高维向量表示，构成了深度语言模型的骨干。这些嵌入是用针对任务的端到端流程优化出来的。[Word2vec](https://en.wikipedia.org/wiki/Word2vec) 就是一个很好的例子。在实践中，它会将语言模型词汇表的 token 从一维的 token 索引映射到 $d$-维的 token 嵌入：

$$\text{Token Index} \in \mathbb{Z}_{0+} \to \text{Token Vector} \in R^{v} \to \text{Token Embedding} \in \mathbb{R}^{d}$$

对一个长度为 $n$ 的 token 序列，这个映射会应用到整个序列上：

$$\text{Token Vector Sequence} \in R^{n\times v} \to \text{Embedding Sequence} \in \mathbb{R}^{n\times d}$$

<br>

### **句子/文档嵌入：捕捉上下文和意义**

在处理句子或整个文档时，嵌入在捕捉上下文、含义和元素之间的交互方面发挥着重要作用。实际上，所有大语言模型都利用类似 Transformer 的架构来生成这些句子/文档的嵌入。

<br>

### **语言生成中的解码器模型**

解码器模型常用于聊天机器人和其它语言生成任务。以 token 序列作为输入，把它们嵌入到隐空间序列中（latent sequence），通过单向推理（unidirectional reasoning）聚焦到输出序列的特定部分。从这个聚焦的、语义密集的点，模型就可以预测序列中的下一个 token：

$$$$
$$\text{[ Next-Token Generation ]}$$
$$\text{Embedding Sequence} \in \mathbb{R}^{n\times d} \to \text{Latent Sequence} \in \mathbb{R}^{n\times d}$$
$$(\text{Latent Sequence})[\text{last entry}] \in \mathbb{R}^{d} \to \text{Token Prediction} \in \mathbb{R}^{v}$$
$$$$

此过程持续进行，通过将 token 预测从向量表示中具象为实际的 token 来构建出一个预测序列，直到满足终止条件，比如达到长度限制或遇到停止 token。

$$$$
$$\text{[ Autoregressive Generation ]}$$
$$(\text{Original + Predicted Embedding Sequence}) \in \mathbb{R}^{(n+1)*e} \to \text{Token Prediction} \in \mathbb{R}^{v}$$
$$\vdots$$
$$(\text{Original + Predicted Embedding Sequence}) \in \mathbb{R}^{(n+m)*e} \to \text{Token Prediction} \in \mathbb{R}^{v}$$
$$$$
<br>

### **用于序列编码的编码器模型**

编码器模型使用双向架构（bidirectional architecture），与解码器模型相比，它们更适用于 token 或序列预测等任务。假设 $c$ 是类的数量或回归值（regressional value）：

$$$$
$$\text{[ Per-Token Prediction ]}$$
$$\text{Embedding Sequence} \in \mathbb{R}^{n\times d} \to \text{Latent Sequence} \in \mathbb{R}^{n\times d} \to \text{Per-Token Predictions} \in \mathbb{R}^{n\times c}$$

$$$$
$$\text{[ Full-Sequence Prediction ]}$$
$$\text{Token Sequence} \in \mathbb{R}^{n\times d} \to \text{Latent Sequence} \in \mathbb{R}^{n\times d}$$
$$(\text{Latent Sequence})[\text{first entry}] \in \mathbb{R}^{d} \to \text{Sequence Prediction} \in \mathbb{R}^{c}$$

<br>

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/encoder-decoder.png" width=1200px/>

----

<br>

## **第 2 部分：** 使用 NVIDIAEmbeddings 模型

在这个 notebook，我们将使用一个有两种路径（pathway）的嵌入模型：一个是针对短格式的 “查询”（query）路径，另一个是针对长格式的 “文档”（passage）路径。我们将在这个过程了解到如何正确地推理其输出，并丰富现有的 LLM 工具！

我们将再次利用 AI Foundation 模型入口及 LangChain 来拉取模型！Notebook 的这一部分遵循 [NVIDIAEmbeddings 官方文档](https://python.langchain.com/docs/integrations/text_embedding/nvidia_ai_endpoints)。

在撰写本文时，用 [NVIDIA Retrieval QA Embedding model](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/ai-foundation/models/nvolve-29k) 可以比较好地完成当前任务，它是一个微调过的 [E5-Large 嵌入模型](https://huggingface.co/intfloat/e5-large)，在`nvolveqa_40k` 列表中。请随时查看可用列表来确认这个模型依然可用。

<br> 

### **识别模型**

在可用的模型中，您应该能找到适合用来区分人类可读段落的嵌入模型。在确定入口之后，创建一个连接到它的 `NVIDIAEmbeddings` 实例，看看它都提供了哪些方法。

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

[m for m in NVIDIAEmbeddings.get_available_models() if "embed" in m.id]

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
# embedder = NVIDIAEmbeddings(model="nvidia/nv-embedqa-mistral-7b-v2")
# embedder = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5")
# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4")
# embedder = NVIDIAEmbeddings(model="snowflake/arctic-embed-l")
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

### **用 API 优化数据嵌入**

使用 LangChain `Embeddings` 与 NVIDIAEmbeddings 模型的接口时，我们注意到有针对**查询**和**文档**进行嵌入这两种路径。它们各自的特点将在检索应用对数据的处理和利用中有所体现：

<br>

#### **查询嵌入（Query Embedding）**
* **目标：**为嵌入简短或提问之类的内容而设计，比如简单的陈述或问题。
* **方法：**利用 `embed_query` 来单独嵌入每个查询。
* **检索中的角色：**相当于“键（key）”，帮助文档检索框架执行搜索或查询。
* **使用模式：**根据需要动态嵌入，与经过预处理的文档嵌入集合不同。

<br>  

#### **文档嵌入**
* **目标：**专为长格式或类似响应的内容定制，包括文档块或段落。
* **方法：**用 `embed_documents` 批量处理文档。
* **检索中的角色：**相当于“值（value）”，表示检索系统中的可搜索内容。
* **使用模式：**通常作为预处理环节大规模嵌入，为将要到来的查询创建文档嵌入库。

<br>

#### **底层的相似性和实际应用**

尽管这两个过程的应用不同，但有一个共同的核心功能：将文本内容处理为语义丰富（semantically rich）的向量化表示。用 `embed_query` 还是 `embed_documents` 取决于内容的性质及其在检索系统中的期望用途。

<br>

#### **用几个“查询”和“文档”的例子试试**

让我们用一组示例查询和文档实际试试。这些示例经过精心挑选，能显示出嵌入模型在通用文本推理上的一些有趣特点。

In [ ]:
# Example queries and documents
queries = [
    "What's the weather like in Rocky Mountains?",
    "What kinds of food is Italy known for?",
    "What's my name? I bet you don't remember...",
    "What's the point of life anyways?",
    "The point of life is to have fun :D"
]

documents = [
    "Komchatka's weather is cold, with long, severe winters.",
    "Italy is famous for pasta, pizza, gelato, and espresso.",
    "I can't recall personal names, only provide information.",
    "Life's purpose varies, often seen as personal fulfillment.",
    "Enjoying life's moments is indeed a wonderful approach.",
]

我们既可以用查询路径也可以用文档路径对这些段落进行编码。这两种方式的语法略有不同。

In [ ]:
%%time
# Embedding the queries
q_embeddings = [embedder.embed_query(query) for query in queries]

# Embedding the documents
d_embeddings = embedder.embed_documents(documents)

有了嵌入之后，我们就可以对结果进行简单的相似性检查，看看对于检索来说结果是否合理。嵌入完成之后，运行下方代码来可视化交叉相似度矩阵。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

def plot_cross_similarity_matrix(emb1, emb2):
    # Compute the similarity matrix between embeddings1 and embeddings2
    cross_similarity_matrix = cosine_similarity(np.array(emb1), np.array(emb2))

    # Plotting the cross-similarity matrix
    plt.imshow(cross_similarity_matrix, cmap='Greens', interpolation='nearest')
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.title("Cross-Similarity Matrix")
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_cross_similarity_matrix(q_embeddings, d_embeddings)
plt.xlabel("Query Embeddings")
plt.ylabel("Document Embeddings")
plt.show()

# queries = [
#     "What's the weather like in Rocky Mountains?",
#     "What kinds of food is Italy known for?",
#     "What's my name? I bet you don't remember...",
#     "What's the point of life anyways?",
#     "The point of life is to have fun :D"]
# documents = [
#     "Komchatka's weather is cold, with long, severe winters.",
#     "Italy is famous for pasta, pizza, gelato, and espresso.",
#     "I can't recall personal names, only provide information.",
#     "Life's purpose varies, often seen as personal fulfillment.",
#     "Enjoying life's moments is indeed a wonderful approach."]

----

<br>

## **第 3 部分：[练习]** 更真实的例子

可以观察到，匹配的输入/输出对有较高的嵌入相似性。值得一提的是，根据编码器模型的收敛情况，查询路径和文档路径有可能存在显著差异：

In [ ]:
plt.figure(figsize=(8, 6))
plot_cross_similarity_matrix(
    q_embeddings,
    [embedder.embed_query(doc) for doc in documents]
)
plt.xlabel("Query Embeddings (of queries)")
plt.ylabel("Query Embeddings (of documents)")
plt.show()

**一般来说**，“双编码器”（bi-encoder）的真正用处在于，哪怕输入格式开始出现大幅偏差，也可以通过让第二个编码器训练得与第一个编码器一致来应对。为说明这点，我们可以将文档变长，再尝试相同的实验。

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

expound_prompt = ChatPromptTemplate.from_template(
    "Generate part of a longer story that could reasonably answer all"
    " of these questions somewhere in its contents: {questions}\n"
    " Make sure the passage only answers the following concretely: {q1}."
    " Give it some weird formatting, and try not to answer the others."
    " Do not include any commentary like 'Here is your response'"
)

###############################################################################################
## BEGIN TODO

expound_chain = (
    ## TODO: flesh out documents into a more verbose form by implementing the expound_chain 
    ##  which takes advantage of the prompt and llm provided above.
    {}
)

longer_docs = []
for i, q in enumerate(queries):
    ## TODO: Invoke the expound_chain pipeline as appropriate
    longer_doc = ""
    pprint(f"\n\n[Query {i+1}]")
    print(q)
    pprint(f"\n\n[Document {i+1}]")
    print(longer_doc)
    pprint("-"*64)
    longer_docs += [longer_doc]

## END TODO
###############################################################################################

----

如果您对长文档的感到满意，请运行以下代码比较嵌入的变化。结果可能是类似的，但确实存在一些有助于改善大规模检索效果的数学优化。不同的双编码器模型也会影响结果，请记住这点。

一般来说，最好先根据编码器的推荐用例选用两个不同的编码器路径，只有当您假设想要比较的东西（在这里就是查询和文档）在形式和模态很相似时，再逐渐转向相同的编码器。需要明确一下，虽然我们测试的模型在两个路径间的差别不大，但它仍然是一个需要考虑的因素。

In [ ]:
## At the time of writing, our embedding model supports up to 2048 tokens...
longer_docs_cut = [doc[:2048] for doc in longer_docs]

q_long_embs = [embedder._embed([doc], model_type='query')[0] for doc in longer_docs_cut]
d_long_embs = [embedder._embed([doc], model_type='passage')[0] for doc in longer_docs_cut]

## The difference for any particular example may be very small.
## We've raised the similarity matrix to the power of 5 to try and spot a difference.
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plot_cross_similarity_matrix(q_embeddings, q_long_embs)
plt.xlabel("Query Embeddings (of queries)")
plt.ylabel("Query Embeddings (of long documents)")

plt.subplot(1, 2, 2)
plot_cross_similarity_matrix(q_embeddings, d_long_embs)
plt.xlabel("Query Embeddings (of queries)")
plt.ylabel("Document Embeddings (of long documents)")
plt.show()

**注意：**要想看一个路径表现截然不同的双编码器组件，可以从 [AI Foundation Models](https://catalog.ngc.nvidia.com/ai-foundation-models) 目录里找找 CLIP 模型。这个双编码器对调和了文本和图片这两种差距更大的模态。

----

<br>

## **第 4 部分：[练习]** 嵌入语意护栏（Semantic Guardrails）

在下一个 notebook 中，我们将开始用更高级的工具，它们将在背后使用我们的嵌入模型。不过现在，我们仍然可以先用原始的方式探索几个重要的概念！

具体来说，我们可以将其用作生产环境中部署模型的一个关键组件：**语义护栏**。可以借助嵌入来过滤掉聊天机器人无法回答的消息（或有害的消息）！

**这个练习放到了 [`64_guardrails.ipynb`](64_guardrails.ipynb)。**

----

## **第 5 部分：** 总结

在本 notebook 结束之前，您应熟悉语义嵌入模型的价值所在了，并能用该模型在数据集中检索相关信息！

### <font color="#76b900">**非常好！**</font>

### **接下来**：
1. **[可选]** 回顾 notebook 顶部的“思考问题”。
2. **[进阶]** 如果您有时间，可以查看并尝试完成 Notebook [`64_guardrails.ipynb`](64_guardrails.ipynb)，其中介绍了语义护栏。